<a href="https://colab.research.google.com/github/sgr1118/deep-learning-from-scratch-3/blob/main/%EC%A0%9C2%EA%B3%A0%EC%A7%80_%EC%9E%90%EC%97%B0%EC%8A%A4%EB%9F%AC%EC%9A%B4_%EC%BD%94%EB%93%9C%EB%A1%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 목표
- 더 복잡한 계산을 수행하는 DeZero 확장
- DeZero 파이썬 패키지로 묶어주기

## 11단계 : 가변 길이 인수(순전파 편)
- 함수는 한 개의 변수가 아닌 여러개의 변수를 입력받기도 한다.
- 가변 길이는 인수(또는 반환값)의 수가 달라질 수 있다는 것

### 11.1 Function 클래스 수정
- 가변 길이 입출력을 표현하려면 변수들을 리스트 또는 튜플을 넣어 처리하는 것이 편하다.
- __call__ 메서드의 인수와 반환값을 리스트로 바꿔본다.

In [1]:
# 추가되어진 코드
class Funtion:
    def __call__(self, inputs):
        xs = [x.data for x in inputs] # input을 리스트로
        ys = self.forward(xs)
        outputs = [Variable(as_array(y)) for y in ys] # output을 리스트로

        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        return outputs

### 11.2 Add 클래스 구현
- Add 클래스를 구현한다. 다만 인수와 반환값이 리스트 또는 튜플이여야한다.
- 아래 코드는 더 자연스럽게 개선이 필요하다.

In [ ]:
# 추가 코드
class Add(Function):
    def forward(self, xs):
        x0, x1 = xs
        y = x0 + x1
        return (y,) # 튜플 형태 반환값, 값이 하나 생략되어도 문제 없다.

xs = [Variable(np.array(2)), Variable(np.array(3))]
f = Add()
ys = f(xs)
y = ys[0]
print(y.data)

In [2]:
cd '/content/drive/MyDrive/Book/deeplearng_basic_3/ch_2_py'

/content/drive/MyDrive/Book/deeplearng_basic_3/ch_2_py


In [4]:
!python step11.py

5


## 12단계 : 가변 길이 인수(개선 편)
- Add 클래스를 이용하는 사람을 위한 개선
- 구현하는 사람을 위한 개선

### 12.1 첫 번째 개선 : 함수를 사용하기 쉽게
- 기존 클래스는 인수를 리스트에 모아서 전달받지만 개선하면 이 같은 과정을 거치지 않아도된다.

In [14]:
# 추가 코드
class Function:
    def __call__(self, *inputs): # 임의 개수의 인수(가변 길이 인수)를 건네 함수를 호출할 수 있다.
        xs = [x.data for x in inputs] # input을 리스트로
        ys = self.forward(xs)
        outputs = [Variable(as_array(y)) for y in ys] # output을 리스트로

        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs

        # 리스트의 원소가 하나라면 첫 번째 원소를 반환한다.
        return outputs if len(outputs) > 1 else outputs[0]

In [17]:
!python step12.py

5


### 12.2 두 번째 개선: 함수를 구현하기 쉽도록
- 기존 Add 클래스는 인수는 리스트로 전달되고 결과는 튜플로 반환했다.
- 개선하면 입력, 변수를 직접 받고 결과도 변수를 직접 돌려준다.

In [ ]:
# 추가 코드
class Function:
    def __call__(self, *inputs): # 임의 개수의 인수(가변 길이 인수)를 건네 함수를 호출할 수 있다.
        xs = [x.data for x in inputs] # input을 리스트로
        ys = self.forward(*xs) # 언팩
        if not isinstance(ys, tuple): # 튜플이 아닌 경우 튜플로 변경
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys] # output을 리스트로

        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs

        # 리스트의 원소가 하나라면 첫 번째 원소를 반환한다.
        return outputs if len(outputs) > 1 else outputs[0]

In [18]:
!python step12.py

5


### 12.3 add 함수 구현
- 마지막으로 Add 클래스를 '파이썬 함수'로 사용할 수 있는 코드 추가

In [ ]:
# 추가 코드
def add(x0, x1):
    return Add()(x0, x1)

In [20]:
!python step12.py

5


## 13단계 : 가변 길이 인수(역전파 편)

### 13.1 가변 길이 인수에 대응한 Add 클래스의 역전파
- 상류에서 온 미분값을 '그대로 흘려보내는 것'이 덧셈의 역전파이다.

In [ ]:
# 추가 코드
class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

    def backward(self, gy): # 입력이 1개, 출력이 2개
        return gy, gy

### 13.2 Variable 클래스 수정
- 앞서 Variable 클래스는 함수의 입출력이 하나라고 가정했으나 이 부분을 여러 개의 변수가 대응할 수 있도록 수정

In [ ]:
# 추가 코드
def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data) # self.data와 형상과
            # 데이터 터입이 같은 ndarray 인스턴스 생성
        funcs = [self.creator]
        while funcs:
            f = funcs.pop() # 1. 함수를 가져온다.
            gys = [output.grad for output in f.outputs] # 출력 변수(미분값)을 리스트에 담는다.
            gxs = f.backward(*gys) # 역전파 호출 : 리스트 언팩
            if not isinstance(gxs, tuple): # 튜플이 아닌 경우 튜플로 변경
                gxs = (gxs, )
            for x, gx in zip(f.inputs, gxs): # 모든 Variable 인스턴스 각각에 알맞은 미분값을 설정
                x.grad = gx

                if x.creator is not None:
                    funcs.append(x.creator)

### 13.3 Square 클래스 구현
- Square 클래스또한 새로운 Variable과 Function에 맞게 수정할 필요가 있다.

In [ ]:
# 추가 코드

class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.inputs[0].data # 복수형 inputs로 변경
        gx = 2 * x * gy
        return gx

    def squqre(x):
        f = Square()
        return f(x)

In [23]:
!python step13.py

13.0
4.0
6.0


- $z = x^2 + y^2$이라는 계산을 z = add(square(x), square(y))라는 코드로 풀어냈다. 그런 다음 z.backward()를 호출하면 미분 계산이 자동으로 이루어진다.

## 14단계: 같은 변수 반복 사용
- 현재까지 코드에서 같은 변수를 반복 사용하면 의도대로 동작하지 않을 수 있다는 문제가 있다.
- 동일한 변수를 사용하면 덧셈을 제대로 미분할 수 없다.

### 14.1 문제의 원인
- step13.py에서 Variable 클래스에 문제가 있다. 현재는 전해지는 미분값을 그대로 대입하고 있다. 즉 같은 변수를 반복해서 사용하면 미분값이 덮어 씌여진다.
- 해결하기 위해 전파되는 미분값의 '합'이 필요하다.

### 14.2 해결책
- 미분값의 '합'을 구하는 코드 구현

In [ ]:
# 코드 수정

class Variable():
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{}은(는) 지원하지 않습니다.'.format(type(data)))
        self.data = data
        self.grad = None # 미분값 저장
        self.creator = None # 인스턴스 변수 추가

    def set_creator(self, func): # 메서드 추가
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data) # self.data와 형상과
            # 데이터 터입이 같은 ndarray 인스턴스 생성
        funcs = [self.creator]
        while funcs:
            f = funcs.pop() # 1. 함수를 가져온다.
            gys = [output.grad for output in f.outputs] # 출력 변수(미분값)을 리스트에 담는다.
            gxs = f.backward(*gys) # 역전파 호출 : 리스트 언팩
            if not isinstance(gxs, tuple): # 튜플이 아닌 경우 튜플로 변경
                gxs = (gxs, )
            for x, gx in zip(f.inputs, gxs): # 모든 Variable 인스턴스 각각에 알맞은 미분값을 설정
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx 

                if x.creator is not None:
                    funcs.append(x.creator)

In [26]:
!python step14.py

2.0
3.0


### 14.3 미분값 재설정
- 문제가 해결된 듯 보이지만 또 다른 문제가 있다. 같은 변수를 사용하여 '다른' 계산을 할 경우 계산이 꼬이는 문제이다.
- 인스턴스를 재사용하는 바람에 두 번째 계산 값에 첫 번째 미분 값이 더해지고 말았다.
- 해결하기 위해 Variable 클래스에 미분값을 초기화하는 cleargead 메서드 추가

In [27]:
!python step14.py

2.0
5.0


In [ ]:
# 추가 코드
def cleargrad(self):
    self.grad = None # 여러 가지 미분을 계산할 때 똑같은 변수를 재사용 가능

In [28]:
!python step14.py

2.0
3.0


## 15단계: 복잡한 계산 그래프(이론 편)
- 현재 우리가 구현한 Dezero는 일직선 계산 그래프를 잘 수행하지만 더 복잡한 계산 그래프의 역전파를 제대로 할 수 없다.

### 15.1 역전파의 올바른 순서

![](https://velog.velcdn.com/images/hyxxnii/post/3e7d8eb2-4eff-442b-84e4-6b8bb79d980c/image.png)
<center>중간에 분기했다가 다시 합류하는 계산 그래프</center>

- 현재 DeZero는 위와 같은 계산 그래프의 역전파도 수행할 수 없다.
- 계산 중간에 'a'라는 변수는 역전파 출력시 미분값이 더해야 한다. 'a'의 미분을 계산하려면 'a'의 출력 쪽에서 전파하는 2개의 미분값이 필요하다.
- 그 2개의 미분값이 전달된 후에야 a > x 전파가 가능하다.

### 15.2 현재의 DeZero
- Variable 클래스에서 funcs 리스트를 자세히봐라.
- while의 마지막 code에서 처리할 함수의 후보를 funcs 리스트의 끝에 추가하고 있다. 그 다음 처리할 함수를 그 리스트의 끝에서 꺼낸다.
- 이렇게 된다면 함수의 처리 순서가 D,C,A,B,A가 되어 버린다.
- 역전파 과정을 다시 생각해보자. funcs 리스트에 D가 추가되어 [D] 상태로 시작된다. 그 다음 D의 입력 변수인 B와 C가 리스트에 추가될 것이다.
- 이 시점에서 funcs 리스트는 [B,C]이다. 그 다음 마지막 원소 C를 꺼내고 A가 리스트에 추가되어 [B,A]가 된다. 그럼 다시 마지막 원소인 A를 꺼내버린다. 원래 B가 나왔어야했다.

In [ ]:
class Variable():
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{}은(는) 지원하지 않습니다.'.format(type(data)))
        self.data = data
        self.grad = None # 미분값 저장
        self.creator = None # 인스턴스 변수 추가

    def set_creator(self, func): # 메서드 추가
        self.creator = func

    def cleargrad(self):
        self.grad = None # 여러 가지 미분을 계산할 때 똑같은 변수를 재사용 가능

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data) # self.data와 형상과
            # 데이터 터입이 같은 ndarray 인스턴스 생성
        funcs = [self.creator]
        while funcs:
            f = funcs.pop() # 1. 함수를 가져온다.
            gys = [output.grad for output in f.outputs] # 출력 변수(미분값)을 리스트에 담는다.
            gxs = f.backward(*gys) # 역전파 호출 : 리스트 언팩
            if not isinstance(gxs, tuple): # 튜플이 아닌 경우 튜플로 변경
                gxs = (gxs, )
            for x, gx in zip(f.inputs, gxs): # 모든 Variable 인스턴스 각각에 알맞은 미분값을 설정
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx # 미분값을 더해준다.

                if x.creator is not None:
                    funcs.append(x.creator)

### 15.3 함수 우선순위
- 이런 문제가 발생한 이유는 함수 우선순위를 설정하지 않았기 때문이다. 우선순위를 설정하는 가장 쉬운 방법은 함수와 변수 간 관계를 활용하는 것이다. 이 관계를 기준으로 함수와 변수의 '세대'를 기록하고 이것이 우선순위인 것이다.

## 16단계: 복잡한 계산 그래프(구현 편)

### 16.1 세대 추가
- Variable, Function 클래스에 인스턴스 변수 generation을 추가한다.

In [ ]:
# 추가 코드

class Variable():
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{}은(는) 지원하지 않습니다.'.format(type(data)))
        self.data = data
        self.grad = None # 미분값 저장
        self.creator = None # 인스턴스 변수 추가
        self.generation = 0 # 세대 수를 기록하는 변수

    def set_creator(self, func): # 메서드 추가
        self.creator = func
        self.generation = func.generation + 1 # 세대를 기록 (부모 세대 + 1)

In [ ]:
# 추가 코드

class Function:
    def __call__(self, *inputs): # 임의 개수의 인수(가변 길이 인수)를 건네 함수를 호출할 수 있다.
        xs = [x.data for x in inputs] # input을 리스트로
        ys = self.forward(*xs) # 언팩
        if not isinstance(ys, tuple): # 튜플이 아닌 경우 튜플로 변경
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys] # output을 리스트로

        # 입력 변수와 같은 값으로 generation 설정
        self.generation = max([x.generation for x in inputs])
        
        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs

        # 리스트의 원소가 하나라면 첫 번째 원소를 반환한다.
        return outputs if len(outputs) > 1 else outputs[0]

### 16.3 Variable 클래스의 backward
- add_func 함수를 backward 메서드 안에 중첩 함수로 정의했다. 중첩 함수는 주로 다음 두 조건을 충족할 때 적합하다.
- 감싸는 메서드(backward 메서드) 안에서만 이용한다.
- 감싸는 메서드(backward 메서드)에 정의된 변수(funcs, seen_set)를 사용해야 한다.

In [ ]:
# 추가 코드
class Variable:

   def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data) # self.data와 형상과
            # 데이터 터입이 같은 ndarray 인스턴스 생성

        funcs = []
        seen_set = set() # 함수의 backward 메서드가 여러 번 불리는 것 방지

        def add_func(f): # 함수 리스트 세대 순으로 정렬
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)

        while funcs:
            f = funcs.pop() # 자동으로 세대가 가장 큰 DeZero 함수를 꺼낸다.
            gys = [output.grad for output in f.outputs] # 출력 변수(미분값)을 리스트에 담는다.
            gxs = f.backward(*gys) # 역전파 호출 : 리스트 언팩
            if not isinstance(gxs, tuple): # 튜플이 아닌 경우 튜플로 변경
                gxs = (gxs, )
            for x, gx in zip(f.inputs, gxs): # 모든 Variable 인스턴스 각각에 알맞은 미분값을 설정
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx # 미분값을 더해준다.

                if x.creator is not None:
                    add_func(x.creator) 

### 16.4 동작 확인

In [30]:
!python step16.py

32.0
64.0


## 17단계: 메모리 관리와 순환 참조
- 성능을 개선할 수 있는 대책을 적용할 예정이다.

### 17.1 메모리 관리
- 파이썬은 필요없어진 객체를 삭제한다. 그렇지만 코드를 제대로 작성하지 않으면 메모리 부족 등의 문제가 발생한다. 특히 신경망에서는 대량의 데이터를 다루기에 메모리 관리를 제대로 하지않으면 실행 시간이 매우 오래걸린다.
- 파이썬 메모리 관리 방식은 두 가지이다. 첫 번째는 참조 수를 세는 방식, 두 번째는 세대를 기준으로 쓸모 없어진 객체를 회수하는 방식이다.

### 17.2 참조 카운트 방식의 메모리 관리
- 